In [1]:
import numpy as np
from scipy.stats import unitary_group
from numpy import linalg as LA
from scipy.linalg import block_diag
from scipy.linalg import logm, expm
import qiskit

In [2]:
#functions
def h(xx):
    x=np.real(xx)
    return -x*np.log2(x)-(1-x)*np.log2(1-x)
def H(v):
    summ=0
    d=len(v)
    for j in range(d):
        summ=summ+h(v[j])
    return summ
#following function rotates gamma0=diag(w) with U, extract blocks wrt Y=(2,2,1), and returns block spectrum v
def vvector(w,U):
    gamma0=np.diag(w)
    Udag=np.conjugate(U.T)
    gamma=np.matmul(U,np.matmul(gamma0,Udag))
    block1=gamma[0:2,0:2]
    block2=gamma[2:4,2:4]
    block1ev=LA.eigvals(block1)
    block2ev=LA.eigvals(block2)
    v=np.array([block1ev[0].real,block1ev[1].real,block2ev[0].real,block2ev[1].real,gamma[4][4].real])
    return(v)

#following function returns unitary epsilon close to identity
def unitaryclosetoid(epsilon):
    M = np.random.rand(d, d)-0.5 + 1j*np.random.rand(d, d)  # a (very basic) random complex matrix
    A = (M - np.conj(M.T)) #make it antihermitian. eigenvalues are like between -3i and 3i...
    deltaU=expm(epsilon*A) #unitary close to identity
    return(deltaU)

In [65]:
#choose w vector
#w=np.array([.2,.5,.5,.9,1.]) #value we discussed on blackboard
w=np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])  
#HERE STUDYING THE FOLLOWING NASTY w. Has two small entries...maybe that prevents conjecture
#otherwise it's a new regime
w=np.array([0.07742762, 0.09840953, 0.72567301, 0.80270683, 0.80799754])
w=np.array([0.2,0.3,0.7,0.7,0.8])
w=np.sort(w)
t=sum(w)
d=len(w)

regime=0
if t/5<(w[1]+w[2])/2:
    vconj=np.sort(np.array([(w[1]+w[2])/2,(w[1]+w[2])/2,(w[0]+w[3]+w[4])/3,(w[0]+w[3]+w[4])/3,(w[0]+w[3]+w[4])/3]))
    Iconj=H(vconj)
    regime=1
    print("1st regime")
elif t/5>(w[2]+w[3])/2:
    vconj=np.sort(np.array([(w[2]+w[3])/2,(w[2]+w[3])/2,(w[0]+w[1]+w[4])/3,(w[0]+w[1]+w[4])/3,(w[0]+w[1]+w[4])/3]))
    Iconj=H(vconj)
    print("2nd regime")
    regime=2
else:
    haveconjecture=False
    print("3rd (identity) regime?")
    vconj=np.array([t/d,t/d,t/d,t/d,t/d])
    Iconj=H(vconj)   
    regime=3

3rd (identity) regime?


In [42]:
#number of global unitaries
m=2000000 #still the bottleneck parameter(?).
#looks like 10 mln is enough to "fall close to global maximum".after that local maximization is efficient (?)

#number of best global unitaries to keep on the 'podium'
mpodium=200
#number of close-to-identity unitaries
mm=5000
#decide how close to identity
epsilon=0.01 #try carefully
#number of close-to-identity unitaries for second local round
mmm=100000
epsilon2=0.00001 #try carefully

#allocate memory for some relevant variables
#gamma0=np.diag(w)
vbest=np.array([w[j] for j in range(d)])
v=np.zeros(d)
Ibest=0.
I=0.
Ubest=np.identity(d,dtype=complex)
U=np.identity(d,dtype=complex)
podium=[[0.,np.identity(d,dtype=complex)] for i in range(mpodium)] #will be filled with couples (I value, U)

#step 1: global optimization
for k in range(m):
    U=unitary_group.rvs(d)
    v=vvector(w,U)
    I=H(v)
    if I>podium[0][0]:
        podium[0][0]=np.copy(I)
        podium[0][1]=np.copy(U)
        #order podium; best (I,U) is given by podium[-1]
        podium = sorted(podium, key=lambda x: x[0])
                               
Ibestbeforelocal=np.copy(podium[-1][0])

#step 2: local optimization for each unitary on the podium
deltaU=np.identity(d,dtype=complex)

#! watch out. some a-posteriori check of UNITARITY OF Ubest should be done, see below.
#Bc we are multiplying more and more unitaries... (how many?)
#the errors add up, as opposed to what happens in the global maximization

for k in range(mm):
    deltaU=unitaryclosetoid(epsilon)
    for l in range(mpodium): #treat  podium couples  (I,U) independently
        U = np.matmul(podium[l][1],deltaU) #unitary close to podium[l][1]
        v=vvector(w,U)
        I=H(v)
        if I>podium[l][0]:
            podium[l][0]=np.copy(I)
            podium[l][1]=np.copy(U)
            
#reorder podium            
podium = sorted(podium, key=lambda x: x[0])
# best among podium

Ibestafterlocal=np.copy(podium[-1][0])
Ibest=np.copy(podium[-1][0])
Ubest=np.copy(podium[-1][1])

count=0
for j in range(mpodium):
    if Ibest-podium[j][0]<0.0001:
        count=count+1
print("Out of ", mpodium, " couples (I,U) on the podium, the first ",count, " couples are close to the winner couple")

#step 3: further local optimization
#assuming the winner Ubest is actually good, one can push the local maximization even further to get more digits
#unitarity check in the end could be more important than before as you multiply more and more unitaries

for k in range(mmm):
    deltaU=unitaryclosetoid(epsilon2)
    U = np.matmul(Ubest,deltaU) #unitary close to Ubest
    v=vvector(w,U)
    I=H(v)
    if I>Ibest:
        Ibest=np.copy(I)
        Ubest=np.copy(U)

Out of  200  couples (I,U) on the podium, the first  1  couples are close to the winner couple


In [43]:

print("estimated Imax value after global minimization: ",Ibestbeforelocal)
print("estimated Imax value after first local minimization: ",Ibestafterlocal)
print("estimated Imax value after second local minimization: ",Ibest)
print("upper bound: ", d*h(t/d))

estimated Imax value after global minimization:  4.969791780579395
estimated Imax value after first local minimization:  4.9768602025421815
estimated Imax value after second local minimization:  4.976890116282002
upper bound:  4.976892194101128


In [54]:
#unitarity check
Ubestdag=np.conjugate(Ubest.T)
np.matmul(Ubest,Ubestdag).round(14)

array([[ 1.e+00+0.j,  0.e+00+0.j,  0.e+00-0.j,  0.e+00-0.j,  0.e+00-0.j],
       [ 0.e+00-0.j,  1.e+00+0.j,  1.e-14+0.j,  0.e+00+0.j, -0.e+00-0.j],
       [ 0.e+00+0.j,  1.e-14-0.j,  1.e+00+0.j, -0.e+00-0.j, -0.e+00-0.j],
       [ 0.e+00+0.j,  0.e+00-0.j, -0.e+00+0.j,  1.e+00+0.j,  0.e+00-0.j],
       [ 0.e+00+0.j, -0.e+00+0.j, -0.e+00+0.j,  0.e+00+0.j,  1.e+00+0.j]])

In [67]:
#check conjectures:
vbest=np.sort(vvector(w,Ubest))
print("vbest: ",vbest.round(6))
print("vconj: ",vconj.round(6))

print("\nEstimated Imax value: ",Ibest)
print("Conjectured Imax value: ",Iconj)
diff=Iconj-Ibest
if diff<0:
    print("\nConjecture is wrong!")
else:
    print("\nConjecture not violated; difference is ", diff)

vbest:  [0.539999 0.54     0.54     0.540001 0.540001]
vconj:  [0.54 0.54 0.54 0.54 0.54]

Estimated Imax value:  4.976892194095824
Conjectured Imax value:  4.976892194101128

Conjecture not violated; difference is  5.304201522449148e-12


In [46]:
print("Finished!")

Finished!


In [47]:
for k in range(mmm):
    deltaU=unitaryclosetoid(epsilon2)
    U = np.matmul(Ubest,deltaU) #unitary close to Ubest
    v=vvector(w,U)
    I=H(v)
    if I>Ibest:
        Ibest=np.copy(I)
        Ubest=np.copy(U)

In [49]:
vbest=np.sort(vvector(w,Ubest))
vbest.round(5)

array([0.54, 0.54, 0.54, 0.54, 0.54])

In [39]:
kaustav=np.array([0.554 , 0.5551, 0.5252, 0.5499, 0.5158])

In [40]:
H(vbest)-H(kaustav)

0.003527542218146884

In [41]:
Ibest

array(4.97656925)

In [55]:
vvector(w,Ubest)

array([0.54000074, 0.53999957, 0.53999924, 0.54000068, 0.53999977])

In [51]:
for k in range(100000):
    deltaU=unitaryclosetoid(0.00001)
    U = np.matmul(Ubest,deltaU) #unitary close to Ubest
    v=vvector(w,U)
    I=H(v)
    if I>Ibest:
        Ibest=np.copy(I)
        Ubest=np.copy(U)

In [57]:
w

array([0.2, 0.3, 0.7, 0.7, 0.8])

In [61]:
a=3
b=0.
b=np.copy(a)
b

array(3)

In [63]:
a=np.array([2,3])
b=a[0]
b=5
a

array([2, 3])

In [68]:
print("vbest: ",vbest)
print("vconj: ",vconj)
print("identity regime up to Imax accuracy ",diff)

vbest:  [0.53999924 0.53999957 0.53999977 0.54000068 0.54000074]
vconj:  [0.54 0.54 0.54 0.54 0.54]
identity regime up to Imax accuracy  5.304201522449148e-12
